**Homework 2:**
Top university rankings


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
topuni_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508409283658'
r = requests.get(topuni_url).json()

In [19]:
type(r)

dict

In [184]:
top_unis = pd.DataFrame.from_dict(r['data'])
top_unis

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,US,253,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,US,94,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,GB,95,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge
5,GB,478,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294654,6,Europe,95.3,5,University of Oxford,/universities/university-oxford
6,GB,365,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294014,7,Europe,94.6,None,UCL (University College London),/universities/ucl-university-college-london
7,GB,356,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294030,8,Europe,93.7,None,Imperial College London,/universities/imperial-college-london
8,US,120,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294536,9,North America,93.5,5,University of Chicago,/universities/university-chicago
9,CH,201,Switzerland,"<a href=""/where-to-study/europe/switzerland/gu...","<img src=""https://www.topuniversities.com/site...",294432,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...


In [190]:
details = pd.DataFrame()
for url, nid in zip(top_unis[:200]['url'].values, top_unis[:200]['nid'].values):
    print(url)

    req = requests.get('https://www.topuniversities.com' + url)
    soup = BeautifulSoup(req.text, 'html.parser')
    req2 = requests.get('https://www.topuniversities.com/qs-profiles/classifications/' + nid).json()

    d = dict()
    d['url'] = url
    try:
        d['total faculty'] = int(soup.find("div", class_="total faculty").find("div", class_="number").string.strip().replace(",", ""))
        d['inter faculty'] = int(soup.find("div", class_="inter faculty").find("div", class_="number").string.strip().replace(",", ""))
        d['Number of students'] = int(soup.find("div", class_="total student").find("div", class_="number").string.strip().replace(",", ""))
        d['Postgraduate students [%]'] = int(soup.find("div", class_="total student").find("div", class_="post").span.string.strip().replace("%", ""))
        d['Undergraduate students [%]'] = int(soup.find("div", class_="total student").find("div", class_="grad").span.string.strip().replace("%", ""))
        d['Total international students'] = int(soup.find("div", class_="total inter").find("div", class_="number").string.strip().replace(",", ""))
        d['Postgraduate international students [%]'] = int(soup.find("div", class_="total inter").find("div", class_="post").span.string.strip().replace("%", ""))
        d['Undergraduate international students [%]'] = int(soup.find("div", class_="total inter").find("div", class_="grad").span.string.strip().replace("%", ""))

        #for s in soup.find("ul", class_="score").strings:
        #    print(s)

        #soup.find("div", id="wurc").find("ul", class_="score")

        score_type = [elem.string.strip() for elem in soup.find("ul", class_="score").find_all("div", class_="criteria")]
        score_value = [float(elem.string) for elem in soup.find("ul", class_="score").find_all("div", class_="text")]
        lis = list(zip(score_type, score_value))

        d.update(dict((type_, value_) for type_, value_ in lis))
        d.update(req2['classifications'])
        #temp = soup.find("ul", id = "uni-info").find_all("h4")

        #d['Age'] = temp[1].span.string
        #d['Status'] = temp[2].span.string
        #d['Research'] = temp[3].span.string
        #d['Focus'] = temp[4].span.string
        #for element in temp:
        #   d[element.contents[0].strip()] = element.span.string.strip()
    except:
        pass
    details = pd.concat([details, pd.DataFrame.from_dict([d])])
    
details.reset_index()

/universities/massachusetts-institute-technology-mit
/universities/stanford-university
/universities/harvard-university
/universities/california-institute-technology-caltech
/universities/university-cambridge
/universities/university-oxford
/universities/ucl-university-college-london
/universities/imperial-college-london
/universities/university-chicago
/universities/eth-zurich-swiss-federal-institute-technology
/universities/nanyang-technological-university-singapore-ntu
/universities/ecole-polytechnique-f%C3%A9d%C3%A9rale-de-lausanne-epfl
/universities/princeton-university
/universities/cornell-university
/universities/national-university-singapore-nus
/universities/yale-university
/universities/johns-hopkins-university
/universities/columbia-university
/universities/university-pennsylvania
/universities/australian-national-university
/universities/duke-university
/universities/university-michigan
/universities/kings-college-london
/universities/university-edinburgh
/universities/tsi

,index,Academic Reputation,Citations per Faculty,Employer Reputation,Faculty Student,International Faculty,International Students,Number of students,Overall Score,Postgraduate international students [%],...,Undergraduate international students [%],Undergraduate students [%],age_value,data_status_value,inter faculty,research_intensity_value,size_value,subject_range_value,total faculty,url
0,0,100.0,99.9,100.0,100.0,100.0,96.1,11067.0,100.0,83.0,...,17.0,40.0,5,B,1679.0,VH,M,CO,2982.0,/universities/massachusetts-institute-technolo...
1,0,100.0,99.4,100.0,100.0,99.6,72.7,15878.0,98.7,83.0,...,17.0,44.0,5,B,2042.0,VH,L,FC,4285.0,/universities/stanford-university
2,0,100.0,99.9,100.0,98.3,96.5,75.2,22429.0,98.4,85.0,...,15.0,31.0,5,B,1311.0,VH,L,FC,4350.0,/universities/harvard-university
3,0,99.5,100.0,85.4,100.0,93.4,89.2,2255.0,97.7,87.0,...,13.0,44.0,5,B,350.0,VH,S,CO,953.0,/universities/california-institute-technology-...
4,0,100.0,78.3,100.0,100.0,97.4,97.7,18770.0,95.6,64.0,...,36.0,63.0,5,A,2278.0,VH,L,FC,5490.0,/universities/university-cambridge
5,0,100.0,76.3,100.0,100.0,98.6,98.5,19720.0,95.3,74.0,...,26.0,56.0,5,A,2964.0,VH,L,FC,6750.0,/universities/university-oxford
6,0,99.7,74.7,99.5,99.1,96.6,100.0,31080.0,94.6,55.0,...,45.0,52.0,5,A,2554.0,VH,XL,FC,6345.0,/universities/ucl-university-college-london
7,0,99.4,68.7,100.0,100.0,100.0,100.0,16090.0,93.7,53.0,...,47.0,57.0,5,A,2071.0,VH,L,FC,3930.0,/universities/imperial-college-london
8,0,99.9,85.9,92.9,96.5,71.9,79.8,13557.0,93.5,76.0,...,24.0,43.0,5,B,635.0,VH,L,FC,2449.0,/universities/university-chicago
9,0,99.6,98.7,99.4,68.2,100.0,98.8,19815.0,93.3,72.0,...,28.0,47.0,5,A,1886.0,VH,L,CO,2477.0,/universities/eth-zurich-swiss-federal-institu...


In [210]:
result = pd.concat([top_unis[:200], details], axis=1, join_axes=[details.url])
result.index = list(range(200))

In [212]:
details.head()

,Academic Reputation,Citations per Faculty,Employer Reputation,Faculty Student,International Faculty,International Students,Number of students,Overall Score,Postgraduate international students [%],Postgraduate students [%],...,Undergraduate international students [%],Undergraduate students [%],age_value,data_status_value,inter faculty,research_intensity_value,size_value,subject_range_value,total faculty,url
0,100.0,99.9,100.0,100.0,100.0,96.1,11067.0,100.0,83.0,60.0,...,17.0,40.0,5,B,1679.0,VH,M,CO,2982.0,/universities/massachusetts-institute-technolo...
0,100.0,99.4,100.0,100.0,99.6,72.7,15878.0,98.7,83.0,56.0,...,17.0,44.0,5,B,2042.0,VH,L,FC,4285.0,/universities/stanford-university
0,100.0,99.9,100.0,98.3,96.5,75.2,22429.0,98.4,85.0,69.0,...,15.0,31.0,5,B,1311.0,VH,L,FC,4350.0,/universities/harvard-university
0,99.5,100.0,85.4,100.0,93.4,89.2,2255.0,97.7,87.0,56.0,...,13.0,44.0,5,B,350.0,VH,S,CO,953.0,/universities/california-institute-technology-...
0,100.0,78.3,100.0,100.0,97.4,97.7,18770.0,95.6,64.0,37.0,...,36.0,63.0,5,A,2278.0,VH,L,FC,5490.0,/universities/university-cambridge


In [214]:
top = top_unis[:200]
top.head()

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,US,253,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,US,94,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,GB,95,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge


In [216]:
Total = top.merge(details, left_on='url', right_on='url', how='outer')

In [220]:
Total.head()

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,...,Total international students,Undergraduate international students [%],Undergraduate students [%],age_value,data_status_value,inter faculty,research_intensity_value,size_value,subject_range_value,total faculty
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,...,3717.0,17.0,40.0,5,B,1679.0,VH,M,CO,2982.0
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,...,3611.0,17.0,44.0,5,B,2042.0,VH,L,FC,4285.0
2,US,253,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294270,3,North America,98.4,5,...,5266.0,15.0,31.0,5,B,1311.0,VH,L,FC,4350.0
3,US,94,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294562,4,North America,97.7,5,...,647.0,13.0,44.0,5,B,350.0,VH,S,CO,953.0
4,GB,95,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294561,5,Europe,95.6,5,...,6699.0,36.0,63.0,5,A,2278.0,VH,L,FC,5490.0
